# 水平集函数的重置化

## (1) 重置化方程
    
1. 时间步长取为：

$$\Delta{t} = \frac{1}{2}\frac{\Delta_{\min}}{\max~\big|(\mathrm{sign}(\Phi_n)_{ij}\big|},\quad\Delta_{\min} = \min(\Delta{x},\Delta{y})$$
    
2. 数值计算时可以通过磨光：
$$\mathrm{sign}(\Phi)=\frac{\Phi}{\sqrt{\Phi^2+|\nabla\Phi|^2(\Delta x)^2}}$$
其中 $\nabla\Phi$ 采用中心差分近似，$D_{ij}^{cx}$ 和 $D_{ij}^{cy}$ 分别是两个维度中的中心差分算子。

3. 计算格式：
$$\Phi_{ij}^{n+1} = \Phi_{ij}^n - \Delta{t}\bigg(\max\big(\mathrm{sign}(\Phi_{ij}^n),0\big)\nabla^{+} + \min\big(\mathrm{sign}(\Phi_{ij}^n),0\big)\nabla^{-} - \mathrm{sign}(\Phi_{ij}^n)\bigg)$$
其中
$$\nabla^{+} = \big[\max(D_{ij}^{-x},0)^2 + \min(D_{ij}^{+x},0)^2 + \max(D_{ij}^{-y},0)^2 + \min(D_{ij}^{+y},0)^2\big]^{1/2}$$
$$\nabla^{-} = \big[\max(D_{ij}^{+x},0)^2 + \min(D_{ij}^{-x},0)^2 + \max(D_{ij}^{+y},0)^2 + \min(D_{ij}^{-y},0)^2\big]^{1/2}$$
这里，$\Delta{t}$ 是时间步长，$D_{ij}^{\pm{x}}$ 和 $D_{ij}^{\pm{y}}$ 分别是 $\bm{x}\in\mathbb{R}^2$ 的两个维度中的向前和向后差分算子。

In [67]:
import numpy as np

def upwind_diff(phi, d, direction):
    """
    使用迎风格式计算向前和向后有限差分.

    Parameters:
    - phi ( ndarray - (nely+2, nlex+2) ): 水平集函数在水平集网格点上的值;
    - d (float): x 或 y 方向上相邻网格的间距, 取决于 'direction' 参数;
    - direction (str): 'x': 沿 x 方向计算差分, 'y': 表示沿 y 方向计算差分.

    Returns:
    - back_diff ( ndarray - (nely+2, nlex+2) ): 向后差分矩阵: ( phi(i, j) - phi(i-1, j) ) / dx
                            或 ( phi(i, j) - phi(i, j-1) ) / dx, 取决于 'direction'.
    - fawd_diff ( ndarray - (nely+2, nlex+2) ): 向前差分矩阵: ( phi(i+1, j) - phi(i, j) ) / dx
                            或 ( phi(i, j+1) - phi(i, j) ) / dx, 取决于 'direction'.
    """
    # 根据指定的方向计算后向和前向差分
    if direction == 'x':
        x_minus_1 = np.roll(phi, 1, axis=1) # x 方向向右位移
        x_plus_1 = np.roll(phi, -1, axis=1) # x 方向向左位移
        dx = d
        back_diff = (phi - x_minus_1) / dx
        fawd_diff = (x_plus_1 - phi) / dx
    elif direction == 'y':
        y_minus_1 = np.roll(phi, 1, axis=0) # y 方向向下位移
        y_plus_1 = np.roll(phi, -1, axis=0) # y 方向向上位移
        dy = d
        back_diff = (phi - y_minus_1) / dy
        fawd_diff = (y_plus_1 - phi) / dy
    else:
        raise ValueError("direction 必须是 'x' 或 'y'")
    
    return back_diff, fawd_diff

def reinit(phi0, dx, dy, loop_num):
    """ 
    将水平集函数重初始化为符号距离函数.

    Parameters:
    - phi0 ( ndarray - (nely+2, nelx+2) ): 重置化前的水平集界面;
    - dx: 有限元单元的宽度;
    - dy: 有限元单元的高度;
    - loop_num: 重初始化的时间步数.

    Returns:
    - sign_dist_phi (ndarray - (nely+2, nelx+2) ): 重置化后的水平集界面.
    """
    for _ in range(loop_num + 1):
        # 水平集函数沿 x 和 y 方向的向前和向后差分算子
        dx_L, dx_R = upwind_diff(phi0, dx, 'x')
        dy_L, dy_R = upwind_diff(phi0, dy, 'y')
        
        # 水平集函数沿 x 和 y 方向的中心差分算子
        dx_C = (dx_L + dx_R) / 2
        dy_C = (dy_L + dy_R) / 2
        
        # sign(Phi) 的数值计算
        signPhi = phi0 / (np.sqrt(phi0**2 + (dx_C**2 + dy_C**2) * dx**2) + np.finfo(float).eps)

        # CFL 时间步长
        detT = 0.5 * min(dx, dy) / np.max(np.abs(signPhi))
        
        grad_plus = np.sqrt(np.maximum(dx_L, 0)**2 + np.minimum(dx_R, 0)**2 +
                            np.maximum(dy_L, 0)**2 + np.minimum(dy_R, 0)**2)
        grad_minus = np.sqrt(np.minimum(dx_L, 0)**2 + np.maximum(dx_R, 0)**2 +
                            np.minimum(dy_L, 0)**2 + np.maximum(dy_R, 0)**2)

        phi0 = phi0 - detT * \
            (np.maximum(signPhi, 0)*grad_plus + np.minimum(signPhi, 0)*grad_minus - signPhi)

    sign_dist_phi = phi0

    return sign_dist_phi

## (2) reinit 函数检验
符号距离函数
$$\phi(x,y) = \sqrt{(x-0.5)^2+(y-0.75)^2} - 0.15$$
其$$(x,y)\in[0, 1]\times[0,1]$。
$$|\nabla\phi| = \sqrt{\bigg(\frac{x-0.5}{\sqrt{(x-0.5)^2+(y-0.5)^2}}\bigg)^2 + \bigg(\frac{y-0.75}{\sqrt{(x-0.5)^2+(y-0.5)^2)}}\bigg)^2} = 1$$
非符号距离函数
$$\psi(x,y) = (x-0.5)^2+(y-0.75)^2 - 0.0115$$
其中$(x,y)\in[0, 1]\times[0,1]$。
$$|\nabla\psi| = \sqrt{(2(x-0.5)^2 + (2(y-0.75))^2} = 2\sqrt{(x-0.5)^2+(y-0.75)^2} = 2\sqrt{0.0115} \neq 1$$

In [68]:
from fealpy.mesh import UniformMesh2d

# 测试参数
nelx, nely = 20, 20
domain = [0, 1, 0, 1]
dw = domain[1] - domain[0]
dh = domain[3] - domain[2]
ew = (domain[1]-domain[0]) / nelx 
eh = (domain[3]-domain[2]) / nely 

from fealpy.mesh import UniformMesh2d
ls_domain = [-ew/2, dw+ew/2, -eh/2, dh+eh/2]
ls_mesh = UniformMesh2d(extent=(0, nelx+1, 0, nely+1), 
                    h=(ew, eh), origin=(ls_domain[0], ls_domain[2]))
ls_NC = ls_mesh.number_of_cells()
print("ls_NC:", ls_NC)
ls_NN = ls_mesh.number_of_nodes()
print("ls_NN:", ls_NN)
ls_node = ls_mesh.entity('node')
print("ls_node:", ls_node.shape, "\n", ls_node)

ls_NC: 441
ls_NN: 484
ls_node: (484, 2) 
 [[-0.025 -0.025]
 [-0.025  0.025]
 [-0.025  0.075]
 [-0.025  0.125]
 [-0.025  0.175]
 [-0.025  0.225]
 [-0.025  0.275]
 [-0.025  0.325]
 [-0.025  0.375]
 [-0.025  0.425]
 [-0.025  0.475]
 [-0.025  0.525]
 [-0.025  0.575]
 [-0.025  0.625]
 [-0.025  0.675]
 [-0.025  0.725]
 [-0.025  0.775]
 [-0.025  0.825]
 [-0.025  0.875]
 [-0.025  0.925]
 [-0.025  0.975]
 [-0.025  1.025]
 [ 0.025 -0.025]
 [ 0.025  0.025]
 [ 0.025  0.075]
 [ 0.025  0.125]
 [ 0.025  0.175]
 [ 0.025  0.225]
 [ 0.025  0.275]
 [ 0.025  0.325]
 [ 0.025  0.375]
 [ 0.025  0.425]
 [ 0.025  0.475]
 [ 0.025  0.525]
 [ 0.025  0.575]
 [ 0.025  0.625]
 [ 0.025  0.675]
 [ 0.025  0.725]
 [ 0.025  0.775]
 [ 0.025  0.825]
 [ 0.025  0.875]
 [ 0.025  0.925]
 [ 0.025  0.975]
 [ 0.025  1.025]
 [ 0.075 -0.025]
 [ 0.075  0.025]
 [ 0.075  0.075]
 [ 0.075  0.125]
 [ 0.075  0.175]
 [ 0.075  0.225]
 [ 0.075  0.275]
 [ 0.075  0.325]
 [ 0.075  0.375]
 [ 0.075  0.425]
 [ 0.075  0.475]
 [ 0.075  0.525]
 [ 0.0

In [69]:
import numpy as np
from fealpy.decorator import cartesian

# 非符号距离函数的水平集函数
@cartesian
def phi_nsd(p):
    x = p[..., 0]
    y = p[..., 1]
    val = (x - 0.5)**2 + (y - 0.75)**2 - 0.0115

    return val

# 符号距离函数的水平集函数
@cartesian
def phi_sd(p):
    x = p[..., 0]
    y = p[..., 1]
    val = np.sqrt((x - 0.5)**2 + (y - 0.75)**2) - 0.15

    return val

lsf_nsd = phi_nsd(ls_node).reshape(nelx+2, nely+2).T
print("lsf_nsd:", lsf_nsd.shape, "\n", lsf_nsd)
lsf_sd = phi_sd(ls_node).reshape(nelx+2, nely+2).T
print("lsf_sd:", lsf_sd.shape, "\n", lsf_sd)

error_nsd_sd = np.max(np.abs(lsf_nsd - lsf_sd))
print("error_nsd_sd:", error_nsd_sd)

lsf_nsd: (22, 22) 
 [[ 8.6475e-01  8.1475e-01  7.6975e-01  7.2975e-01  6.9475e-01  6.6475e-01
   6.3975e-01  6.1975e-01  6.0475e-01  5.9475e-01  5.8975e-01  5.8975e-01
   5.9475e-01  6.0475e-01  6.1975e-01  6.3975e-01  6.6475e-01  6.9475e-01
   7.2975e-01  7.6975e-01  8.1475e-01  8.6475e-01]
 [ 7.8975e-01  7.3975e-01  6.9475e-01  6.5475e-01  6.1975e-01  5.8975e-01
   5.6475e-01  5.4475e-01  5.2975e-01  5.1975e-01  5.1475e-01  5.1475e-01
   5.1975e-01  5.2975e-01  5.4475e-01  5.6475e-01  5.8975e-01  6.1975e-01
   6.5475e-01  6.9475e-01  7.3975e-01  7.8975e-01]
 [ 7.1975e-01  6.6975e-01  6.2475e-01  5.8475e-01  5.4975e-01  5.1975e-01
   4.9475e-01  4.7475e-01  4.5975e-01  4.4975e-01  4.4475e-01  4.4475e-01
   4.4975e-01  4.5975e-01  4.7475e-01  4.9475e-01  5.1975e-01  5.4975e-01
   5.8475e-01  6.2475e-01  6.6975e-01  7.1975e-01]
 [ 6.5475e-01  6.0475e-01  5.5975e-01  5.1975e-01  4.8475e-01  4.5475e-01
   4.2975e-01  4.0975e-01  3.9475e-01  3.8475e-01  3.7975e-01  3.7975e-01
   3.8475e-01

In [70]:
maxit = 10
errorType = ['$|| sd - nsd_reinit ||_{\\infty}$']
errorMatrix = np.zeros((1, maxit), dtype=np.float64)
for i in range(maxit):
    print("The {}-th computation:".format(i))
    lsf_nsd_reinit = reinit(phi0=lsf_nsd, dx=ew, dy=eh, loop_num=(10*(i+1)))
    errorMatrix[0, i] = np.max(np.abs(lsf_sd - lsf_nsd_reinit))

    for i, errType in enumerate(errorType):
        print(errType)
        print(errorMatrix[i])
        print('-------')

The 0-th computation:
$|| sd - nsd_reinit ||_{\infty}$
[0.41682691 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
-------
The 1-th computation:
$|| sd - nsd_reinit ||_{\infty}$
[0.41682691 0.39862034 0.         0.         0.         0.
 0.         0.         0.         0.        ]
-------
The 2-th computation:
$|| sd - nsd_reinit ||_{\infty}$
[0.41682691 0.39862034 0.39389376 0.         0.         0.
 0.         0.         0.         0.        ]
-------
The 3-th computation:
$|| sd - nsd_reinit ||_{\infty}$
[0.41682691 0.39862034 0.39389376 0.39087649 0.         0.
 0.         0.         0.         0.        ]
-------
The 4-th computation:
$|| sd - nsd_reinit ||_{\infty}$
[0.41682691 0.39862034 0.39389376 0.39087649 0.38880186 0.
 0.         0.         0.         0.        ]
-------
The 5-th computation:
$|| sd - nsd_reinit ||_{\infty}$
[0.41682691 0.39862034 0.39389376 0.39087649 0.38880186 0.38729978
 0.         0.         0.         0.  

In [71]:
import os
visualization_dir = 'visualization/'
os.makedirs(visualization_dir, exist_ok=True)
fname = os.path.join(visualization_dir, 'wang_reinit.vts')
ls_mesh.to_vtk(filename=fname, 
               nodedata={'lsf_nsd': lsf_nsd.flatten('F'),
                         'lsf_nsd_reinit': lsf_nsd_reinit.flatten('F'),
                         'lsf_sd': lsf_sd.flatten('F')})

'visualization/wang_reinit.vts'